In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from pathlib import Path

import sys

sys.path.append('../src')

from Config.config import PATHS

import warnings
warnings.filterwarnings("ignore")

### Test focal region similarity

In [ ]:
from Classes.focal_regions import FocalRegion

In [ ]:
region = np.array([[1, 0, 1], [1, 1, 0], [0, 1, 1]])
fra = FocalRegion(region)
fra.debug = True
print('Focal region:')
print(fra)

# history = np.array([[1, 0], [0, 1], [0, 1]])
history = np.array([[0], [1], [1]])
print('History:\n', history)
print('-'*60)
fra.get_action_preferences(history, 0)

In [ ]:
file_name = Path('..', 'LaTeX', 'FRA_region_1.png')
FocalRegion.draw_region(region, file=file_name)

In [ ]:
file_name = Path('..', 'LaTeX', 'history_2.png')
FocalRegion.draw_region(history, file=file_name)

### Test Focal Regions

In [2]:
from Classes.focal_regions import SetFocalRegions

In [14]:
sfr = SetFocalRegions(
    num_agents=2,
    threshold=0.5,
    len_history=2,
    c=0.9,
    steepness=20,
    max_regions=4,
)

In [15]:
sfr.generate_focal_regions()
print(sfr)

Region 0
------------------------------------------------------------
[[0. 1.]
 [1. 0.]]
------------------------------------------------------------
Region 1
------------------------------------------------------------
[[0. 1.]
 [1. 0.]]
------------------------------------------------------------
Region 2
------------------------------------------------------------
[[1. 1.]
 [0. 0.]]
------------------------------------------------------------
Region 3
------------------------------------------------------------
[[0. 0.]
 [1. 1.]]
------------------------------------------------------------



In [ ]:
history = np.array([1, 0, 0])
sfr.add_history(history)
history = np.array([0, 1, 1])
sfr.add_history(history)
print('History')
print(sfr.history)
sfr.debug = True
for region in sfr.focal_regions:
    region.debug = True
sfr.get_action_preferences(agent_id=0)

In [ ]:
sfr.normalized_logistic(np.array([0.83, 0.17]))

In [ ]:
X = np.linspace(0, 1, 1000)
Y = sfr.normalized_logistic(X)

fig, ax = plt.subplots(1, 1, figsize=(3,3))
sns.lineplot(x=X, y=Y, ax=ax)
ax.axvline(x=sfr.c, color='red', linestyle='--', linewidth=1)
ax.set_title(f'Normalized logistic')
ax.grid()

fig.savefig(PATHS['LaTeX'] / 'logistic.png', dpi=300)

### Test FRA agents

In [ ]:
from Classes.bar import Bar
from Utils.interaction import Episode
from Classes.focal_regions import SetFocalRegions
from Classes.cognitive_model_agents import FocalRegionAgent

In [ ]:
free_parameters = {
    'inverse_temperature': 6,
    'len_history': 2,
}
fixed_parameters = {
    'num_agents': 2,
    'threshold': 0.5,
}
n = 1

In [ ]:
# Create the environment
bar = Bar(
    num_agents=fixed_parameters['num_agents'],
    threshold=fixed_parameters['threshold']
)
# Create the agents
agents = [
    FocalRegionAgent(
        free_parameters, 
        fixed_parameters, 
        n
    ) for n in range(fixed_parameters['num_agents'])
]
# Create the episode
episode = Episode(
    model='FRA',
    agents=agents,
    environment=bar,
    num_rounds=13
)

episode.run()

bar.render()

# Test simulations

In [ ]:
from Utils.interaction import Performer
from Utils.utils import GetMeasurements
from Classes.cognitive_model_agents import FRAplus

In [ ]:
fixed_parameters = {
	"num_agents":2,
	"threshold":0.5,
}
free_parameters = {
    'inverse_temperature': 6, 
    'bias': 0.75, 
    'learning_rate': 0.2, 
    'len_history': 2, 
    'delta': 0.4
}
simulation_parameters = {
	'num_rounds': 50,
    'num_episodes': 23,
	'verbose':False
}

In [ ]:
df = Performer.sim(
    agent_class=FocalRegionAgent,
    fixed_parameters=fixed_parameters,
    free_parameters=free_parameters,
    simulation_parameters=simulation_parameters
)
gm = GetMeasurements(
    data=df, measures=['efficiency', 'inequality']
)
data = gm.get_measurements()
ax = sns.scatterplot(
    x='inequality', y='efficiency',
    data=data
)
ax.set_xlim([-0.05, 0.6])
ax.set_ylim([-0.05, .55])

In [ ]:
sns.lineplot(
    x='round', y='score',
    data=df
)

# Test parameter fit

In [ ]:
from Classes.cognitive_model_agents import FRAplus
from Classes.parameter_recovery import ParameterFit

In [ ]:
# my_models = [FocalRegionAgent]
my_models = [FRAplus]

best_fit_file_2P = PATHS['parameter_fit_results'] / 'best_fit_2P_3.json'

#Load 2P data
file = PATHS['human_data'] / '2-player-UR.csv'
print(f'Loading data from {file}...')
data = pd.read_csv(file)
data.head(2)

In [ ]:
hyperparameters = {
    'n_iter': 128,
    'init_points': 64,
}

ParameterFit.run(
    data=data,
    model_list=my_models,
    best_fit_path=best_fit_file_2P,
    optimizer_type='bayesian',
    hyperparameters=hyperparameters,
    new_file=False
)

In [ ]:
ParameterFit.run(
    data=data,
    model_list=my_models,
    best_fit_path=best_fit_file_2P,
    optimizer_type='scipy',
    hyperparameters={},
    new_file=False
)

{'inverse_temperature': np.float64(5.939793200426039), 'bias': np.float64(0.7908615558239465), 'learning_rate': np.float64(0.18051110357718222), 'len_history': np.float64(2.4724091615619512), 'c': np.float64(1.0), 'delta': np.float64(0.1)}
Deviance: -2181.1293794903972
AIC: 4374.2587589807945

In [ ]:
from Classes.parameter_recovery import GetDeviance

free_parameters = {
    'inverse_temperature': 6, 
    'bias': 0.79, 
    'learning_rate': 0.18, 
    'len_history': 2, 
    'c': 0.1, 
    'delta': 0.1
}
g_dev = GetDeviance(
    model=FRAplus,
    free_parameters=free_parameters,
    data=data
)

g_dev.get_deviance_from_data(free_parameters)

---